# POC-English-Amb Tests `2016-05-31+06-27`
3rd test (with v.0.5 hierarchical generalization) of POC-English-Amb-2018-05-31.ipynb
    
Each line is calculated 5x, parsing metrics tested 2x for each calculation.  
The detailed calculation table is shared as 'long_table.txt' in data folder  
[http://langlearn.singularitynet.io/data/clustering_2018/POC-English-Amb-2018-05-31+06-27/](http://langlearn.singularitynet.io/data/clustering_2018/POC-English-Amb-2018-05-31+06-27/)  
This notebook is shared as static html via  
[http://langlearn.singularitynet.io/data/clustering_2018/html/POC-English-Amb-2018-05-31+06-27.html](http://langlearn.singularitynet.io/data/clustering_2018/html/POC-English-Amb-2018-05-31+06-27.html)  
Previous tests:  
[http://langlearn.singularitynet.io/data/clustering_2018/html/POC-English-Amb-2018-05-31.html](http://langlearn.singularitynet.io/data/clustering_2018/html/POC-English-Amb-2018-05-31.html)  
[http://langlearn.singularitynet.io/data/clustering_2018/html/POC-English-Amb-2018-05-31+06-06.html](http://langlearn.singularitynet.io/data/clustering_2018/html/POC-English-Amb-2018-05-31+06-06.html)

## Basic settings

In [1]:
import os, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)
src_path = module_path + '/src'
if os.path.exists(src_path) and src_path not in sys.path: sys.path.append(src_path)
lg_path = '/home/oleg/miniconda3/envs/ull4/lib/python3.6/site-packages/linkgrammar'
if os.path.exists(lg_path) and lg_path not in sys.path: sys.path.append(lg_path)
from src.utl.utl import UTC
from src.utl.read_files import check_dir
from src.utl.turtle import html_table
from src.grammar_learner.poc05 import learn_grammar, params, parse_metrics, run_learn_grammar
prefix = ''
tmpath = module_path + '/tmp/'
check_dir(tmpath, True, 'none')
print(UTC(), ':: module_path =', module_path)
start = time.time()

2018-06-27 06:26:41 UTC :: module_path = /home/oleg/language-learning


## Grammar Learner parameters

In [2]:
out_dir = module_path + '/output/POC-English-Amb-05-31+' + str(UTC())[:10]
corpus = 'POC-English-Amb'
dataset = 'MST_fixed_manually'
kwargs = {
    'left_wall'     :   'LEFT-WALL' ,
    'period'        :   True        ,
    'context'       :   2           ,
    'word_space'    :   'discrete'  ,
    'clustering'    :   'group'     ,
    'grammar_rules' :   2           ,
    'verbose'       :   'min'       ,
    'tmpath'        :   tmpath      , 
    'categories_generalization': 'off',
    'rules_generalization': 'off',
    'test_corpus'   : module_path + '/data/POC-English-Amb/poc_english.txt',
    'reference_path': module_path + '/data/POC-English-Amb/poc-english_ex-parses-gold.txt',
    'template_path' : 'poc-turtle', 
    'linkage_limit' : 1 }

## Integration test

In [3]:
%%capture
response = run_learn_grammar(corpus, dataset, module_path, out_dir, **kwargs)

In [4]:
print('Parse-ability / parse-quality = ', str(response['parse_ability']) + '% / ' \
    + str(response['parse_quality'])+'%; ', response['grammar_rules'], 'Link Grammar rules')

Parse-ability / parse-quality =  12% / 6%;  39 Link Grammar rules


## Test function

In [5]:
def run_test(**kwargs):
    spaces = ''
    if kwargs['context'] == 1: 
        spaces += 'c'
    else: spaces += 'd'
    if kwargs['word_space'] == 'vectors': 
        spaces += 'DRK'
    else: spaces += 'ILE'
    if kwargs['grammar_rules'] == 1: 
        spaces += 'c'
    else: spaces += 'd'
    details = []
    average = []
    for dataset in ['MST_fixed_manually', 'LG_English', 'R=6_Weight=6:R_mst-weight=1:R']:
        for kwargs['left_wall'] in ['LEFT-WALL', '']:
            if kwargs['left_wall'] == 'LEFT-WALL': 
                lw = 'LW'
                kwargs['period'] = True
                dot = ' + '
            else: 
                lw = ' -- '
                dot = ' -- '
                kwargs['period'] = False
            for g in [('',''), ('jaccard',''), ('jaccard','jaccard')]:
                if lw == 'LW' and dot == ' + ' and g != ('',''): continue
                elif kwargs['grammar_rules'] == 1 and g[0] != '': continue
                elif kwargs['word_space'] == 'vectors' and g[1] != '': continue
                elif kwargs['context'] == 1 and g[1] != '': continue
                kwargs['rules_generalization'] = g[0]
                kwargs['categories_generalization'] = g[1]
                if g[0] == '' and g[1] == '': gen = 'none'
                elif g[0] == 'jaccard' and g[1] == '': gen = 'rules'
                elif g[0] == '' and g[1] == 'jaccard': gen = 'categories'
                else: gen = 'both'
                ip, oc, og = params(corpus, dataset, module_path, out_dir, **kwargs)
                pa = []
                pq = []
                rules = []
                for j in range(5):
                    r = learn_grammar(ip, oc, og, **kwargs)
                    for i in range(2):
                        a, q, lg_parse_path = parse_metrics(r['grammar_file'], **kwargs)
                        pa.append(a)
                        pq.append(q)                            
                        rules.append(r['grammar_rules'])
                        rulestr = ' ' + str(r['grammar_rules']) + ' '
                        line = [corpus, dataset, lw, dot, gen, spaces, rulestr, \
                            str(a)+'%', str(q)+'%', str(int(round(a*q/100, 0)))+'%']
                        details.append(line)
                paa = int(round(sum(pa)/len(pa), 0))
                pqa = int(round(sum(pq)/len(pq), 0))
                rules_avg = int(round(sum(rules)/len(rules), 0))
                avg = [corpus, dataset, lw, dot, gen, spaces, rules_avg, \
                    str(paa)+'%', str(pqa)+'%', str(int(round(paa*pqa/100, 0)))+'%']
                average.append(avg)
    return average, details

# POC-English-Amb, no generalization, 3 datasets
ULL Project Plan ⇒ Parses ⇒ lines 14-22

## Connectors-DRK-Connectors

In [6]:
%%capture
kwargs['context'] = 1
kwargs['word_space'] = 'vectors'
kwargs['clustering'] = 'kmeans'
kwargs['grammar_rules'] = 1
average21, long21 = run_test(**kwargs)
table = average21
long_table = long21
#table = []
#long_table = []

In [7]:
header = ['Corpus','Parsing','LW','"."','Generalization','Space','Rules','PA','PQ','PQA']
display(html_table([header]+average21))

Corpus,Parsing,LW,""".""",Generalization,Space,Rules,PA,PQ,PQA
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,27,67%,36%,24%
POC-English-Amb,MST_fixed_manually,--,--,none,cDRKc,28,71%,62%,44%
POC-English-Amb,LG_English,LW,+,none,cDRKc,27,78%,42%,33%
POC-English-Amb,LG_English,--,--,none,cDRKc,28,79%,64%,51%
POC-English-Amb,R=6_Weight=6:R_mst-weight=1:R,LW,+,none,cDRKc,29,85%,51%,43%
POC-English-Amb,R=6_Weight=6:R_mst-weight=1:R,--,--,none,cDRKc,25,81%,48%,39%


## Connectors-DRK-Disjuncts

In [8]:
%%capture
kwargs['context'] = 1
kwargs['word_space'] = 'vectors'
kwargs['clustering'] = 'kmeans'
kwargs['grammar_rules'] = 2
average22, long22 = run_test(**kwargs)
table.extend(average22)
long_table.extend(long22)

In [9]:
display(html_table([header]+average22))

Corpus,Parsing,LW,""".""",Generalization,Space,Rules,PA,PQ,PQA
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKd,29,39%,16%,6%
POC-English-Amb,MST_fixed_manually,--,--,none,cDRKd,28,41%,38%,16%
POC-English-Amb,MST_fixed_manually,--,--,rules,cDRKd,26,4%,4%,0%
POC-English-Amb,LG_English,LW,+,none,cDRKd,29,26%,12%,3%
POC-English-Amb,LG_English,--,--,none,cDRKd,30,35%,30%,10%
POC-English-Amb,LG_English,--,--,rules,cDRKd,27,3%,3%,0%
POC-English-Amb,R=6_Weight=6:R_mst-weight=1:R,LW,+,none,cDRKd,27,78%,45%,35%
POC-English-Amb,R=6_Weight=6:R_mst-weight=1:R,--,--,none,cDRKd,25,70%,41%,29%
POC-English-Amb,R=6_Weight=6:R_mst-weight=1:R,--,--,rules,cDRKd,23,16%,9%,1%


## Disjuncts-DRK-Disjuncts

In [10]:
%%capture
kwargs['context'] = 2
kwargs['word_space'] = 'vectors'
kwargs['clustering'] = 'kmeans'
kwargs['grammar_rules'] = 2
average23, long23 = run_test(**kwargs)
table.extend(average23)
long_table.extend(long23)

In [11]:
display(html_table([header]+average23))

Corpus,Parsing,LW,""".""",Generalization,Space,Rules,PA,PQ,PQA
POC-English-Amb,MST_fixed_manually,LW,+,none,dDRKd,31,17%,9%,2%
POC-English-Amb,MST_fixed_manually,--,--,none,dDRKd,5,79%,58%,46%
POC-English-Amb,MST_fixed_manually,--,--,rules,dDRKd,5,78%,58%,45%
POC-English-Amb,LG_English,LW,+,none,dDRKd,3,94%,43%,40%
POC-English-Amb,LG_English,--,--,none,dDRKd,16,38%,33%,13%
POC-English-Amb,LG_English,--,--,rules,dDRKd,16,1%,1%,0%
POC-English-Amb,R=6_Weight=6:R_mst-weight=1:R,LW,+,none,dDRKd,35,73%,41%,30%
POC-English-Amb,R=6_Weight=6:R_mst-weight=1:R,--,--,none,dDRKd,3,95%,63%,60%
POC-English-Amb,R=6_Weight=6:R_mst-weight=1:R,--,--,rules,dDRKd,3,95%,62%,59%


## Disjuncts-ILE-Disjuncts

In [12]:
%%capture
kwargs['context'] = 2
kwargs['word_space'] = 'discrete'
kwargs['clustering'] = 'group'
kwargs['grammar_rules'] = 2
average24, long24 = run_test(**kwargs)
table.extend(average24)
long_table.extend(long24)

In [13]:
display(html_table([header]+average24))

Corpus,Parsing,LW,""".""",Generalization,Space,Rules,PA,PQ,PQA
POC-English-Amb,MST_fixed_manually,LW,+,none,dILEd,39,12%,6%,1%
POC-English-Amb,MST_fixed_manually,--,--,none,dILEd,37,36%,33%,12%
POC-English-Amb,MST_fixed_manually,--,--,rules,dILEd,28,2%,2%,0%
POC-English-Amb,MST_fixed_manually,--,--,both,dILEd,28,42%,38%,16%
POC-English-Amb,LG_English,LW,+,none,dILEd,39,6%,3%,0%
POC-English-Amb,LG_English,--,--,none,dILEd,37,27%,23%,6%
POC-English-Amb,LG_English,--,--,rules,dILEd,31,0%,0%,0%
POC-English-Amb,LG_English,--,--,both,dILEd,31,28%,24%,7%
POC-English-Amb,R=6_Weight=6:R_mst-weight=1:R,LW,+,none,dILEd,40,65%,35%,23%
POC-English-Amb,R=6_Weight=6:R_mst-weight=1:R,--,--,none,dILEd,38,54%,33%,18%


## All calculations

In [14]:
display(html_table([header]+long_table))

Corpus,Parsing,LW,""".""",Generalization,Space,Rules,PA,PQ,PQA
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,29,67%,36%,24%
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,29,67%,36%,24%
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,28,67%,36%,24%
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,28,67%,36%,24%
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,30,67%,36%,24%
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,30,67%,36%,24%
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,25,67%,36%,24%
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,25,67%,36%,24%
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,22,67%,36%,24%
POC-English-Amb,MST_fixed_manually,LW,+,none,cDRKc,22,67%,36%,24%


In [15]:
from src.utl.write_files import list2file
table_str = list2file(table, out_dir+'/short_table.txt')
long_table_str = list2file(long_table, out_dir+'/long_table.txt')

In [16]:
print(UTC(), ':: finished, elapsed', str(int(round((time.time()-start)/60, 0))), 'min')

2018-06-27 06:30:20 UTC :: finished, elapsed 4 min
